<a href="https://colab.research.google.com/github/pinemoon01/ITB-2022/blob/main/2016343_%EB%B0%95%EC%A7%80%ED%98%84_%EB%B6%84%EB%A5%98%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline
import pandas as pd
import seaborn as sns           # Seaborn 로딩하기
import matplotlib.pyplot as plt # Matplotlib의 pyplot 로딩하기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#마이너스 폰트가 깨지는 문제를 대처하기
plt.rc("axes", unicode_minus = False)

In [ ]:
#한글폰트 설정

!pip install koreanize-matplotlib
import koreanize_matplotlib

In [ ]:
#csv 데이터 가져오기
hd = pd.read_csv("/content/drive/My Drive/ITB/health_data.csv", encoding='UTF-8')

In [ ]:
#먼저 데이터에 대한 전반적인 정보를 확인합니다. 
display(hd.head())
print(hd.info())

In [ ]:
hd.describe()

In [ ]:
# 의사결정나무 만드는 과정
!pip install mglearn  


In [ ]:
import mglearn

In [ ]:
!pip install --upgrade joblib==1.1.0

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [ ]:
hd2 = hd.dropna()

의사결정나무
독립변수는 X에 해당하는 변수들로 설정하였고
종속변수는 음주변수로 설정하였습니다. 음주여부에 따라서 체중, 허리둘레 그리고 수축기혈압의 변화를 분석하고 싶었습니다.


In [ ]:
X = hd2[['체중(5Kg 단위)','허리둘레','수축기혈압','음주여부']]
y = hd2['음주여부']

print(X.head())
print(y.head())

In [ ]:
#결측치 모두 제거

X_drop_row = X.dropna(axis = 0)
print("결측치 모두 제거:",X_drop_row.shape)

In [ ]:
X = X_drop_row.iloc[:,0:2]
y = X_drop_row["음주여부"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state=42)
tree = DecisionTreeClassifier(random_state=0)  
tree.fit(X_train, y_train)
print("학습용 데이터 정확도: {:.3f}".format(tree.score(X_train, y_train)))
print("시험용 데이터 정확도: {:.3f}".format(tree.score(X_test, y_test)))



In [ ]:
tree = DecisionTreeClassifier(max_depth = 4, random_state=0)  
tree.fit(X_train, y_train)
print("학습용 데이터 정확도: {:.3f}".format(tree.score(X_train, y_train)))
print("시험용 데이터 정확도: {:.3f}".format(tree.score(X_test, y_test)))



In [ ]:
#의사결정나무
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(tree, out_file="tree.dot", class_names= ["No","Yes"], feature_names= X_train.columns, impurity=False, filled=True)

with open("tree.dot") as f:
    dot_graph = f.read()

display(graphviz.Source(dot_graph))

In [ ]:
#앙상블 모델
#랜덤 포레스트

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# radius_mean, texture_mean 변수만 이용하여 
# 5개의 나무를 만드는 예 (n_estimators = 5, 실제는 수백개의 트리를 만든다)
forest = RandomForestClassifier(n_estimators=5, random_state=2)
forest.fit(X_train.iloc[:, 0:2], y_train)

# 5개 나무 각각의 분류 경계를 그림 (서로 모양이 많이 다르다)
# 2차원으로 나무의 결과를 보여주기 위한 코드로 mglearn의 함수들을 이용 - 여기서는 설명 생략
fig, axes = plt.subplots(2, 3, figsize=(20, 10))
for i, (ax, tree) in enumerate(zip(axes.ravel(), forest.estimators_)):
    ax.set_title("Tree {}".format(i))
    mglearn.plots.plot_tree_partition(X.iloc[:, 0:2].values, y, tree, ax=ax)
    
mglearn.plots.plot_2d_separator(forest, X.iloc[:, 0:2].values, fill=True, ax=axes[-1, -1], alpha=.4)
axes[-1, -1].set_title("Random forest")
mglearn.discrete_scatter(X.iloc[:, 0], X.iloc[:, 1], y)
plt.show()

In [ ]:
forest = RandomForestClassifier(n_estimators=100, random_state=0)
forest.fit(X_train, y_train)

print("학습용 데이터 세트 정확도: {:.3f}".format(forest.score(X_train, y_train)))
print("시험용 데이터 세트 정확도: {:.3f}".format(forest.score(X_test, y_test)))

In [ ]:
#랜덤포레스트와 의사결정나무 비교
from sklearn.model_selection import cross_val_score

# 교차 검증을 10번 수행하여 10번의 교차 검증 평균 정확도를 비교 (10-fold cross validation)
# default cv=5

dt_scores = cross_val_score(tree, X_train, y_train, cv=10, scoring='accuracy')  
rf_scores = cross_val_score(forest, X_train, y_train, cv=10, scoring='accuracy')
#cv가 교차검증 

print("Accuracy")
print("Decision tree: ", dt_scores) 
print("Random forest: ", rf_scores) 

print("Accuracy mean")
print("Decision tree :{:.3f}".format(dt_scores.mean())) 
print("Random forest :{:.3f}".format(rf_scores.mean())) 

In [ ]:
cv_list = [    
            ['decision_tree',dt_scores],
            ['random_forest',rf_scores],
          ]
df = pd.DataFrame.from_dict(dict(cv_list))
df.plot()

In [ ]:
from sklearn import preprocessing
X = preprocessing.StandardScaler().fit(X).transform(X)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

from sklearn import tree
tree_model = tree.DecisionTreeClassifier(criterion='entropy', max_depth=5)
tree_model.fit(X_train, y_train)
y_hat = tree_model.predict(X_test)

from sklearn import metrics
matrix = metrics.confusion_matrix(y_test, y_hat)
print(matrix)

report = metrics.classification_report(y_test, y_hat)
print(report)

In [ ]:
#그레디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
# 기본값은 max_depth= 3, n_estimators=100, learning_rate = 0.1

gbrt = GradientBoostingClassifier(random_state=0)
gbrt.fit(X_train, y_train)

In [ ]:
print("학습용 데이터 세트 정확도: {:.3f}".format(gbrt.score(X_train, y_train)))
print("시험용 데이터 세트 정확도: {:.3f}".format(gbrt.score(X_test, y_test)))

In [ ]:
#학습용 데이터 세트의 정확도가 73퍼센트이기 때문에 과적합은 아닙니다


주요 변수 설명


In [ ]:
#주요 변수 설명
result = tree.feature_importances_
print("변수의 중요도:\n", result)
#0은 중요 X, 1에 가까울수록 중요 

In [ ]:
import numpy as np

def plot_feature_importances_hd2(model):
    plt.rcParams["figure.figsize"] = (9,6)
    n_features = len(X_train.columns[0:]) 
    plt.barh(np.arange(n_features), model.feature_importances_, align='center') 
    plt.yticks(np.arange(n_features), X_train.columns[0:]) 
    plt.xlabel("feature importance")
    plt.ylabel("feature")
    plt.ylim(-1, n_features) 

plot_feature_importances_hd2(tree)